In [1]:
import pandas as pd

In [3]:
A = pd.read_csv('cs_new_total.csv',encoding = 'utf-8-sig')

In [8]:
B = A[['cosing_simliarity']].copy()

In [9]:
B.columns = ['cosine_similarity']

In [10]:
B

,cosine_similarity
0,0.838633
1,0.838508
2,0.888553
3,0.818725
4,0.813184
...,...
30034,0.878179
30035,0.726583
30036,0.878095
30037,0.807682


In [1]:
import tensorflow as tf
import bert
import os
import MeCab
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tqdm import tqdm
from korpat_tokenizer import Tokenizer
from sklearn.metrics.pairwise import cosine_similarity#for similarity

import numpy as np
from numpy import dot
from numpy.linalg import norm

import re

direct_path = 'C:/Users/USER/Documents/Jupyter Notebook/AI_IMEN/korpatbert_v1.0'

os.environ['TF_KERAS'] = '1' # Keras Tensorflow 설정
config_path = direct_path + "/pretrained/korpat_bert_config.json" # KorpatBert Config 파일 경로
vocab_path = direct_path + "/pretrained/korpat_vocab.txt" # KorpatTokenizer Vocabulary 파일 경로
checkpoint_path = direct_path + "/pretrained/model.ckpt-381250" # KorpatBert 모델파일 경로
pretreind_model_dir = direct_path + "/pretrained/" # KorpatBert 모델 디렉토리 경로

dataset_path = direct_path + '/lm_test_data.tsv'

# Hugging face에서 모델 import (특허관련 모델 upload함)

In [2]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import random
import pandas as pd
import torch#pytorch
from transformers import AutoTokenizer, AutoModel#for embeddings
from sklearn.metrics.pairwise import cosine_similarity#for similarity
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("Pray123/first", use_auth_token=True)

model = AutoModelForPreTraining.from_pretrained("Pray123/first", use_auth_token=True)

In [3]:
all_data = pd.read_csv('블록체인_2_상세_청구항.csv',index_col = 0)

a = [''.join(pd.Series(i).dropna().tolist()).replace('삭제','') for i in all_data[all_data.columns[-180:]].values]

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
all_data['요약'] = a

In [7]:
all_data['요약']

0       블록체인 시스템이 개시된다. 일 실시 예에 따른 블록체인 시스템은 분산 네트워크를 ...
1       블록체인과 IPFS 기반의 그룹별 데이터 공유 시스템이 제공된다. 상기 블록체인과 ...
2       블록체인과 IPFS 기반의 암호화 데이터 공유 시스템이 제공된다. 상기 블록체인과 ...
3       본 발명에 따른 블록체인을 활용한 개인정보 보호 시스템은 하나의 개인정보가 다수개로...
4       블록체인 및 에스크로 기반 크라우드펀딩 서비스 제공 방법이 제공되며, 프로젝트 제안...
                              ...                        
8579    본 발명은 테스팅 동안에 다기능 집적 회로 내의 버스 경합을 방지하는 시스템에 관한...
8580    본 발명은 스캔 셀(scan cell)에 관한 것으로서, 특히 칩에 내장된 내부 메...
8581    높은 정도의 동기화 정확도를 갖는 라인 대 라인(line-to-line) 동기화를 ...
8582    본 발명은, 하수관로에 직립되게 시공되는 복수 개의 기둥부재; 복수 개의 상기 기둥...
8583    배관용 압착체결구를 개시한다. 이러한 배관용 압착체결구는, 관체의 이음부 둘레와 대...
Name: 요약, Length: 8584, dtype: object

In [8]:
tokenizer_mecab = Tokenizer(vocab_path=direct_path + "/pretrained/korpat_vocab.txt", cased=True)

In [9]:

tokenizer_mecab.tokenize('블록체인과 머신러닝')
tokenizer_mecab.encode('블록체인과 머신러닝',max_len= 10)

([5, 500, 9840, 55, 4685, 20446, 20848, 6, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [14]:

def get_embeddings_mecab(text,token_length):
    pattern = r'\([^)]*\)'
    text =  re.sub(pattern=pattern, repl='', string= text)
    
    a = tokenizer_mecab.encode(text,max_len = token_length)
    tokens = {'input_ids' : a[0] , 'attention_mask' : [1 if i !=0 else 0 for i in a[0]]}

    output=model(torch.tensor(tokens['input_ids']).unsqueeze(0),
               attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))['logits']
    return output.detach().numpy()

In [297]:
pd.Series(tokenizer_mecab.tokenize(text_test)).value_counts()

상기      40
의       26
,       25
하       24
에       18
항목      16
는       15
을       15
블록      15
트랜잭션    14
dtype: int64

In [301]:
pd.Series(tokenizer.encode(text_test)).value_counts()

4681     40
2715     25
7        25
2620     21
2535     19
         ..
2815      1
11272     1
2709      1
4847      1
3         1
Length: 116, dtype: int64

In [79]:
text_test

'블록체인 시스템이 개시된다. 일 실시 예에 따른 블록체인 시스템은 분산 네트워크를 통하여 상호간에 통신을 수행하고, 상기 블록체인 시스템 외부의 블록체인 클라이언트 장치와 통신을 수행하는 복수의 노드들을 포함하고, 상기 복수의 노드들 각각은, 상기 블록체인 시스템 상에서 발생되는 트랜잭션을 처리하고, 상기 트랜잭션에 포함되고 상기 복수의 노드들 중 어느 하나의 노드에 대응되는 계정 주소를 포함하는 투표 항목을 수집하고, 및 상기 수집된 투표 항목에 기초하여 상기 복수의 노드들 중 적어도 하나의 노드를 블록 생성자로 선출하도록 설정될 수 있다. 이 외에도 명세서를 통해 파악되는 다양한 실시 예가 가능하다.블록체인 시스템에 있어서,분산 네트워크를 통하여 상호간에 통신을 수행하고, 상기 블록체인 시스템 외부의 블록체인 클라이언트 장치와 통신을 수행하는 복수의 컴퓨팅 장치들을 포함하고,상기 복수의 컴퓨팅 장치들 각각은,상기 블록체인 시스템 상에서 발생되는 트랜잭션에 포함된 투표 항목과 수수료 항목을 기초로 상기 복수의 컴퓨팅 장치들 중 적어도 하나의 컴퓨팅 장치를 블록 생성자로 결정하도록 설정되고, 상기 투표 항목은 상기 복수의 컴퓨팅 장치들 중 어느 하나의 컴퓨팅 장치에 대응되는 계정 주소를 포함하고,상기 수수료 항목은 상기 트랜잭션이 발생시키는 트랜잭션 수수료를 나타내고,상기 복수의 컴퓨팅 장치들 각각은,상기 수수료 항목의 상기 트랜잭션 수수료에 비례하는 가중치를 상기 투표 항목에 부여하고,상기 가중치가 부여된 상기 투표 항목을 수집하여 상기 투표 항목에 포함된 상기 계정 주소에 대한 트랜잭션 투표의 양을 연산하고,누적된 상기 트랜잭션 투표의 양을 기초로 상기 트랜잭션 투표의 양이 많은 순서로 상기 계정 주소를 정렬하여, 상위에 랭크된 미리 정해진 수의 적어도 하나의 계정 주소에 대응하는 상기 적어도 하나의 컴퓨팅 장치를 상기 블록 생성자로 결정하도록 설정된, 블록체인 시스템.제1항에 있어서,상기 수수료 항목의 상기 트랜잭션 수수료가 징수되지 않는 경우, 상기 

In [16]:
cosine_similarity(get_embeddings_mecab(all_data['요약'][0], 256) , get_embeddings_mecab(all_data['요약'][1],256))

array([[0.80376387]], dtype=float32)

In [17]:
all_data['요약'][0]

'블록체인 시스템이 개시된다. 일 실시 예에 따른 블록체인 시스템은 분산 네트워크를 통하여 상호간에 통신을 수행하고, 상기 블록체인 시스템 외부의 블록체인 클라이언트 장치와 통신을 수행하는 복수의 노드들을 포함하고, 상기 복수의 노드들 각각은, 상기 블록체인 시스템 상에서 발생되는 트랜잭션을 처리하고, 상기 트랜잭션에 포함되고 상기 복수의 노드들 중 어느 하나의 노드에 대응되는 계정 주소를 포함하는 투표 항목을 수집하고, 및 상기 수집된 투표 항목에 기초하여 상기 복수의 노드들 중 적어도 하나의 노드를 블록 생성자로 선출하도록 설정될 수 있다. 이 외에도 명세서를 통해 파악되는 다양한 실시 예가 가능하다.블록체인 시스템에 있어서,분산 네트워크를 통하여 상호간에 통신을 수행하고, 상기 블록체인 시스템 외부의 블록체인 클라이언트 장치와 통신을 수행하는 복수의 컴퓨팅 장치들을 포함하고,상기 복수의 컴퓨팅 장치들 각각은,상기 블록체인 시스템 상에서 발생되는 트랜잭션에 포함된 투표 항목과 수수료 항목을 기초로 상기 복수의 컴퓨팅 장치들 중 적어도 하나의 컴퓨팅 장치를 블록 생성자로 결정하도록 설정되고, 상기 투표 항목은 상기 복수의 컴퓨팅 장치들 중 어느 하나의 컴퓨팅 장치에 대응되는 계정 주소를 포함하고,상기 수수료 항목은 상기 트랜잭션이 발생시키는 트랜잭션 수수료를 나타내고,상기 복수의 컴퓨팅 장치들 각각은,상기 수수료 항목의 상기 트랜잭션 수수료에 비례하는 가중치를 상기 투표 항목에 부여하고,상기 가중치가 부여된 상기 투표 항목을 수집하여 상기 투표 항목에 포함된 상기 계정 주소에 대한 트랜잭션 투표의 양을 연산하고,누적된 상기 트랜잭션 투표의 양을 기초로 상기 트랜잭션 투표의 양이 많은 순서로 상기 계정 주소를 정렬하여, 상위에 랭크된 미리 정해진 수의 적어도 하나의 계정 주소에 대응하는 상기 적어도 하나의 컴퓨팅 장치를 상기 블록 생성자로 결정하도록 설정된, 블록체인 시스템.제1항에 있어서,상기 수수료 항목의 상기 트랜잭션 수수료가 징수되지 않는 경우, 상기 

In [58]:

def get_embeddings(text,token_length):
    pattern = r'\([^)]*\)'
    text =  re.sub(pattern=pattern, repl='', string= text).replace('\n','')
    
    tokens=tokenizer(text,max_length=token_length,padding='max_length',truncation=True)
    output=model(torch.tensor(tokens.input_ids).unsqueeze(0),
               attention_mask=torch.tensor(tokens.attention_mask).unsqueeze(0))['logits']
    return output.detach().numpy()

In [20]:
tqdm.pandas()

In [22]:
def calculate_similarity(text1,text2,token_length=512):
    out1=get_embeddings(text1,token_length=token_length)#create embeddings of text
    out2=get_embeddings(text2,token_length=token_length)#create embeddings of text

    sim1= cosine_similarity(out1,out2)[0][0]

    print("input한 문장의 관계는 다음과 같습니다 Cosin 1 : {}".format(sim1))


In [23]:
text1 = '본 발명은 블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템에 관한 것으로, 이 시스템은, 상기 프랜차이즈 유통 및 관리 시스템은 프랜차이즈 협력업체 단말과 통신가능하도록 구성된 중개 관리 서버를 포함하고, 협력업체 단말은 프랜차이즈 본점측 단말, 가맹점측 단말, 물류측 단말, 및 금융사측 단말을 포함하고, 중개 관리 서버는 상품의 주문을 관리하는 주문관리를 수행하는 주문관리 모듈, 물류를 관리하는 수행하는 물류관리 모듈, 구매/생산 관리를 수행하는 구매/생산관리 모듈, 재고/손익을 관리하는 재고/손익 관리 모듈, 거래 내역을 블록체인 네트워크에 저장하는 거래내역 등재 모듈을 포함하고, 블록체인 네트워크는 거래내역이 저장되는 블록체인 주노드 네트워크와 블록체인 주노드 네트워크에 저장된 거래내역이 실시간으로 동기화되어 저장되는 블록체인 부노드 네트워크를 포함하고, 중개 관리 서버는 블록체인 주노드 네트워크에 연결되어 있고, 상기 협력업체 단말은 블록체인 부노드 네트워크에 연결되어, 중개 관리 서버는 블록체인 협력업체 사이에 거래가 발생되면 주노드 네트워크에 거래 내역을 저장하는 동시에 부노드 네트워크에 거래 내역을 복제하여 분산 저장하도록 구성된다.'


In [24]:
text2 = """
하나 이상의 반응물이 제1 기간 동안 반응기에서 가열 요소와 열 접촉하도록 흐른다. 가열 사이클의 제1 부분 동안, 하나 이상의 열화학 반응이 개시되도록 가열 요소로 가열함으로써 하나 이상의 반응물에 제1 온도가 제공된다. 상기 하나 이상의 열화학 반응은 열분해, 열발산, 합성, 수소화, 탈수소화, 가수소분해, 또는 이들의 임의의 조합을 포함한다. 제1 가열 요소는 줄 가열에 의해 작동하고 기체가 통과할 수 있도록 하는 다공성 구조를 갖는다. 상기 가열 사이클의 제2 부분 동안, 하나 이상의 반응물은 예를 들어 가열 요소의 전원을 차단함으로써 제1 온도보다 낮은 제2 온도로 제공된다. 제1 기간의 지속 시간은 5초 미만인 가열 사이클의 지속 시간 이상이다."""

In [36]:
calculate_similarity(all_data['요약'][0],all_data['요약'][100])

input한 문장의 관계는 다음과 같습니다 Cosin 1 : 0.9226692318916321


In [31]:
calculate_similarity(text1,text2)

input한 문장의 관계는 다음과 같습니다 Cosin 1 : 0.5463907122612


In [37]:
df_test = pd.DataFrame(all_data[['요약']],columns= ['요약'])

In [38]:
df_test

,요약
0,블록체인 시스템이 개시된다. 일 실시 예에 따른 블록체인 시스템은 분산 네트워크를 ...
1,블록체인과 IPFS 기반의 그룹별 데이터 공유 시스템이 제공된다. 상기 블록체인과 ...
2,블록체인과 IPFS 기반의 암호화 데이터 공유 시스템이 제공된다. 상기 블록체인과 ...
3,본 발명에 따른 블록체인을 활용한 개인정보 보호 시스템은 하나의 개인정보가 다수개로...
4,"블록체인 및 에스크로 기반 크라우드펀딩 서비스 제공 방법이 제공되며, 프로젝트 제안..."
...,...
8579,본 발명은 테스팅 동안에 다기능 집적 회로 내의 버스 경합을 방지하는 시스템에 관한...
8580,"본 발명은 스캔 셀(scan cell)에 관한 것으로서, 특히 칩에 내장된 내부 메..."
8581,높은 정도의 동기화 정확도를 갖는 라인 대 라인(line-to-line) 동기화를 ...
8582,"본 발명은, 하수관로에 직립되게 시공되는 복수 개의 기둥부재; 복수 개의 상기 기둥..."


In [39]:
all_data[['요약']].iloc[0].values

array(['블록체인 시스템이 개시된다. 일 실시 예에 따른 블록체인 시스템은 분산 네트워크를 통하여 상호간에 통신을 수행하고, 상기 블록체인 시스템 외부의 블록체인 클라이언트 장치와 통신을 수행하는 복수의 노드들을 포함하고, 상기 복수의 노드들 각각은, 상기 블록체인 시스템 상에서 발생되는 트랜잭션을 처리하고, 상기 트랜잭션에 포함되고 상기 복수의 노드들 중 어느 하나의 노드에 대응되는 계정 주소를 포함하는 투표 항목을 수집하고, 및 상기 수집된 투표 항목에 기초하여 상기 복수의 노드들 중 적어도 하나의 노드를 블록 생성자로 선출하도록 설정될 수 있다. 이 외에도 명세서를 통해 파악되는 다양한 실시 예가 가능하다.블록체인 시스템에 있어서,분산 네트워크를 통하여 상호간에 통신을 수행하고, 상기 블록체인 시스템 외부의 블록체인 클라이언트 장치와 통신을 수행하는 복수의 컴퓨팅 장치들을 포함하고,상기 복수의 컴퓨팅 장치들 각각은,상기 블록체인 시스템 상에서 발생되는 트랜잭션에 포함된 투표 항목과 수수료 항목을 기초로 상기 복수의 컴퓨팅 장치들 중 적어도 하나의 컴퓨팅 장치를 블록 생성자로 결정하도록 설정되고, 상기 투표 항목은 상기 복수의 컴퓨팅 장치들 중 어느 하나의 컴퓨팅 장치에 대응되는 계정 주소를 포함하고,상기 수수료 항목은 상기 트랜잭션이 발생시키는 트랜잭션 수수료를 나타내고,상기 복수의 컴퓨팅 장치들 각각은,상기 수수료 항목의 상기 트랜잭션 수수료에 비례하는 가중치를 상기 투표 항목에 부여하고,상기 가중치가 부여된 상기 투표 항목을 수집하여 상기 투표 항목에 포함된 상기 계정 주소에 대한 트랜잭션 투표의 양을 연산하고,누적된 상기 트랜잭션 투표의 양을 기초로 상기 트랜잭션 투표의 양이 많은 순서로 상기 계정 주소를 정렬하여, 상위에 랭크된 미리 정해진 수의 적어도 하나의 계정 주소에 대응하는 상기 적어도 하나의 컴퓨팅 장치를 상기 블록 생성자로 결정하도록 설정된, 블록체인 시스템.제1항에 있어서,상기 수수료 항목의 상기 트랜잭션 수수료가 징수되지 않는 

In [66]:
text1 = """본 발명은 블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템에 관한 것으로, 이 시스템은, 상기 프랜차이즈 유통 및 관리 시스템은 프랜차이즈 협력업체 단말과 통신가능하도록 구성된 중개 관리 서버를 포함하고, 협력업체 단말은 프랜차이즈 본점측 단말, 가맹점측 단말, 물류측 단말, 및 금융사측 단말을 포함하고, 중개 관리 서버는 상품의 주문을 관리하는 주문관리를 수행하는 주문관리 모듈, 물류를 관리하는 수행하는 물류관리 모듈, 구매/생산 관리를 수행하는 구매/생산관리 모듈, 재고/손익을 관리하는 재고/손익 관리 모듈, 거래 내역을 블록체인 네트워크에 저장하는 거래내역 등재 모듈을 포함하고, 블록체인 네트워크는 거래내역이 저장되는 블록체인 주노드 네트워크와 블록체인 주노드 네트워크에 저장된 거래내역이 실시간으로 동기화되어 저장되는 블록체인 부노드 네트워크를 포함하고, 중개 관리 서버는 블록체인 주노드 네트워크에 연결되어 있고, 상기 협력업체 단말은 블록체인 부노드 네트워크에 연결되어, 중개 관리 서버는 블록체인 협력업체 사이에 거래가 발생되면 주노드 네트워크에 거래 내역을 저장하는 동시에 부노드 네트워크에 거래 내역을 복제하여 분산 저장하도록 구성된다.
블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템에 있어서,

프랜차이즈 협력업체 단말과 통신가능하도록 구성된 중개 관리 서버를 포함하고,

상기 협력업체 단말은 프랜차이즈 본점측 단말, 가맹점측 단말, 물류측 단말, 및 금융사측 단말을 포함하고,

상기 중개 관리 서버는 상품의 주문을 관리하는 주문관리를 수행하는 주문관리 모듈, 물류를 관리하는 수행하는 문류관리 모듈, 구매/생산 관리를 수행하는 구매/생산관리 모듈, 재고/손익을 관리하는 재고/손익 관리 모듈, 거래 내역을 블록체인 네트워크에 저자하는 거래내역 등재 모듈을 포함하고,

상기 블록체인 네트워크는 거래내역이 저장되는 블록체인 주노드 네트워크와 블록체인 주노드 네트워크에 저장된 거래내역이 실시간으로 동기화되어 저장되는 블록체인 부노드 네트워크를 포함하고,

상기 중개 관리 서버는 블록체인 주노드 네트워크에 연결되어 있고, 상기 협력업체 단말은 블록체인 부노드 네트워크에 연결되어, 중개 관리 서버는 블록체인 협력업체 사이에 거래가 발생되면 주노드 네트워크에 거래 내역을 저장하는 동시에 부노드 네트워크에 거래 내역을 복제하여 분산 저장하도록 구성된 것을 특징으로 하는

블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템.

제1항에 있어서,

가맹점측 단말로부터 중개관리 서버를 통해 본점측 단말에 물품 구매 요청이 발생되면, 중개관리 서버는 금융사측 단말에 가상계좌 개설을 요청하고, 중개관리 서버가 금융사측 단말로부터 가성계좌 개설을 통보받으면 개설된 가상계좌를 가맹점측 단말에 전송하도록 구성된

블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템.

제2항에 있어서,

가맹점측 단말에서 개설된 가상계좌에 입금을 수행하면 금융사측 단말은 입금 확인을 중개 관리 서버에 전송하고, 중개 관리 서버는 본점측 단말에 금융사측으로부터의 입금 확인을 전달한 후, 입금 거래 내역을 블록체인 주노드에 저장하고 주노드에 저장된 입급 거래 내역을 금융사측 단말의 블록체인 부노드, 가맹점측 단말의 블록체인 부노드, 본점측 단말의 블록체인 부노드에 복제하여 분산 저장하도록 구성된 것을 특징으로 하는

블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템.

제1항에 있어서,

중개 관리 서버는 상품의 수요를 예측하기 위한 수요 예측 모듈을 더 포함하고, 상기 수요 예측 모듈은,

가맹점측 단말로부터 월, 일자, 요일, 상품 정보를 포함하는 판매 상품 데이터를 수집하는 데이터 수집부;

수집된 판매 상품 데이터를 월별, 일자별, 요일별 판매 상품 항목으로 분류하는 데이터 분류부;

분류된 데이터를 분석 정의 데이터에 부합하여 분석하여 분석 데이터를 생성하는 데이터 분석부 - 분석 정의 데이터는 분석실행 방식, 분석수행 주기, 분석대상 평가항목 또는 가중치 중에 어느 하나를 포함하며, 분석 데이터는 판매 품목 대한 빈도수를 월별, 요일별, 일자별, 시간대별 수치화한 데이터를 포함하여 구성됨 - ;

분석 데이터를 딥러닝을 통한 머신러닝을 수행하는 데이터 학습부; 및

데이터 학습부로부터 인가받은 머신러닝 결과에 기반하여 월별, 일자별, 요일별, 시간대별, 품목 판매 예측값을 제공하는 수요 예측부를 포함하는

블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템."""

In [67]:
text1 = text1

def calculate_similarity_row(row):
    text0 = text1
    token_length=512
    text2 = row['요약']
    
    out1=get_embeddings(text0,token_length=token_length)#create embeddings of text
    out2=get_embeddings(text2,token_length=token_length)#create embeddings of text

    sim1= cosine_similarity(out1,out2)[0][0]

#     print("input한 문장의 관계는 다음과 같습니다 Cosin 1 : {}".format(sim1))
    
    return sim1

In [68]:
df_100 = df_test[0:100]

In [69]:
lst_data= []
tqdm.pandas()
lst_data = df_test.iloc[0:10].progress_apply(calculate_similarity_row, axis = 1)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:25<00:00,  2.55s/it]


In [70]:
lst_data

0    0.933347
1    0.939691
2    0.949059
3    0.943254
4    0.943850
5    0.892339
6    0.926447
7    0.937567
8    0.946989
9    0.954016
dtype: float32

In [52]:
df_100['sim'] = lst_data

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
df_100.sort_values('sim')

,요약,sim
36,본 발명은 블록체인 기반의 광고 방법 및 그를 수행하기 위한 시스템에 관한 것이다....,0.051530
20,KYC 데이터와 생체인증정보를 보유한 스마트 디바이스를 활용한 블록체인 개인키 생성...,0.431069
17,퍼블릭 블록체인과 프라이빗 블록체인을 연동한 투자 시스템이 제공된다. 상기 퍼블릭 ...,0.734569
46,"본 발명은 블록체인 기반의 인공지능 서비스 플랫폼에 관한 것으로서, 보다 상세하게는...",0.740181
5,작업 증명이나 복수의 기여 노드를 필요로 하지 않고 블록체인에 대한 신뢰를 창출할 ...,0.863774
...,...,...
63,블록체인 기반의 NFT를 이용하는 플랫폼의 서버에서 수행되는 NFT에 대한 위변조를...,0.931605
92,블록체인 기반의 DID를 이용하는 플랫폼을 위한 서비스 제공 장치에서 수행되는 서비...,0.931867
62,"본 발명은 블록체인 기반 암호화폐를 이용한 결제 방법에 관한 것으로, 본 발명에 따...",0.935009
10,블록체인 시스템이 개시된다. 일 실시 예에 따른 블록체인 시스템은 분산 네트워크를 ...,0.944016


In [292]:
tokenizer_mecab.tokenize(text_3)

['[CLS]',
 '발명',
 '하수관',
 '직립',
 '시공',
 '복수',
 '기둥',
 '##부재',
 '[',
 'UN',
 '##K',
 ']',
 '복수',
 '상기',
 '기둥',
 '##부재',
 '지지',
 '하수관',
 '안착',
 '베이스',
 '[',
 'UN',
 '##K',
 ']',
 '상기',
 '베이스',
 '양측',
 '방향',
 '연속',
 '하수관',
 '측면',
 '연장',
 '안착',
 '복수',
 '확장',
 '[',
 'UN',
 '##K',
 ']',
 '상기',
 '베이스',
 '상면',
 '적',
 '층',
 '하수관',
 '상부',
 '연장',
 '복수',
 '[',
 'UN',
 '##K',
 ']',
 '상기',
 '확장',
 '상면',
 '적',
 '층',
 '하수관',
 '상부',
 '연장',
 '상기',
 '양측',
 '방향',
 '연결',
 '복수',
 '[',
 'UN',
 '##K',
 ']',
 '상기',
 '기둥',
 '##부재',
 '상기',
 '베이스',
 '지지',
 '상기',
 '베이스',
 '상기',
 '기둥',
 '##부재',
 '걸림',
 '연결부',
 '[',
 'UN',
 '##K',
 ']',
 '상기',
 '베이스',
 '상기',
 '확장',
 '상기',
 '또는',
 '상기',
 '하수관',
 '통과',
 '하수',
 '의',
 '해',
 '부유',
 '방지',
 '부유',
 '포함',
 '특징',
 '한다',
 '하수관',
 '직립',
 '시공',
 '복수',
 '기둥',
 '##부재',
 '[',
 'UN',
 '##K',
 ']',
 '복수',
 '상기',
 '기둥',
 '##부재',
 '지지',
 '하수관',
 '안착',
 '베이스',
 '[',
 'UN',
 '##K',
 ']',
 '상기',
 '베이스',
 '양측',
 '방향',
 '연속',
 '하수관',
 '측면',
 '연장',
 '안착',
 '복수',
 '확장',
 '[',
 'UN',
 '##K',
 

In [74]:
all_data.iloc[36]

(51) Int. CL                             ['G06Q30/02(2012.01.01)']
(52) CPC         ['G06Q30/0273(2013.01)', 'G06Q30/0248(2013.01)...
(21) 출원번호/일자                            1020200169966 (2020.12.07)
(71) 출원인                                                ['주식회사핀업']
(11) 등록번호/일자                                                   NaN
                                       ...                        
175                                                            NaN
176                                                            NaN
177                                                            NaN
178                                                            NaN
113 TYPE=SOFT                                                  NaN
Name: 36, Length: 208, dtype: object

# 최종 정리

In [367]:
# 불용어 사전
stopwords = pd.read_csv('stopwords.csv',encoding = 'cp949',header= None)


In [403]:

def get_embeddings(text,token_length ):
    pattern = r'\([^)]*\)'
    text_2 =  re.sub(pattern=pattern, repl='', string= text).replace('\n','')


    lst_all = []
    for j in [i if len(i) != 0 else  '' for i in tokenizer.tokenize(text_2)]:
        if (len(j) == 0) | ('#' in j) | ('상기' in j) | ('[' in j) | (j in stopwords[0].values) | (',' in j):
            pass
        else:
            lst_all.append(j)

    text_3 = ' '.join(lst_all)

    tokens=tokenizer(text_3 ,max_length=token_length,padding='max_length',truncation=True)

    output=model(torch.tensor(tokens['input_ids']).unsqueeze(0),
               attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))['logits']
    return output.detach().numpy()



def get_embeddings_mecab(text,token_length):
    pattern = r'\([^)]*\)'
    text_2 =  re.sub(pattern=pattern, repl='', string= text).replace('\n','')


    lst_all = []
    for j in [i if len(i) != 0 else  '' for i in tokenizer.tokenize(text_2)]:
        if (len(j) == 0) | ('#' in j) | ('상기' in j) | ('[' in j) | (j in stopwords[0].values) | (',' in j):
            pass
        else:
            lst_all.append(j)

    text_3 = ' '.join(lst_all)
    
    a = tokenizer_mecab.encode(text_3,max_len = token_length)
    tokens = {'input_ids' : a[0] , 'attention_mask' : [1 if i !=0 else 0 for i in a[0]]}

    output=model(torch.tensor(tokens['input_ids']).unsqueeze(0),
               attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))['logits']
    return output.detach().numpy()

t
text1 = text1

def calculate_similarity_row(row):
    text0 =  df_test.iloc[461].values[0] # 기준이 되는 text를 넣으면 된다.!!!!
    token_length=1024
    text2 = row['요약']
    
    out1=get_embeddings(text0,token_length=token_length)#create embeddings of text
    out2=get_embeddings(text2,token_length=token_length)#create embeddings of text

    sim1= cosine_similarity(out1,out2)[0][0]

#     print("input한 문장의 관계는 다음과 같습니다 Cosin 1 : {}".format(sim1))
    
    return sim1

def calculate_similarity_mecab_row(row):
    text0 =  df_test.iloc[461].values[0] # 기준이 되는 text를 넣으면 된다.!!!!
    token_length=512
    text2 = row['요약']
    
    out1=get_embeddings_mecab(text0,token_length=token_length)#create embeddings of text
    out2=get_embeddings_mecab(text2,token_length=token_length)#create embeddings of text

    sim1= cosine_similarity(out1,out2)[0][0]

#     print("input한 문장의 관계는 다음과 같습니다 Cosin 1 : {}".format(sim1))
    
    return sim1

In [438]:
result_data = df_test.iloc[lst_IPC].progress_apply(calculate_similarity_mecab_row, axis =1)

100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [01:47<00:00,  2.84s/it]


In [405]:
df_test.iloc[8580:].progress_apply(calculate_similarity_mecab_row, axis =1)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.93s/it]


8580    0.897371
8581    0.777730
8582    0.860893
8583    0.890858
dtype: float32

In [415]:
all_data.columns[:31]

Index(['(51) Int. CL', '(52) CPC ', '(21) 출원번호/일자', '(71) 출원인', '(11) 등록번호/일자',
       '(65) 공개번호/일자', '(11) 공고번호/일자', '(86) 국제출원번호/일자', '(87) 국제공개번호/일자',
       '(30) 우선권정보', '법적상태', '심사진행상태', '심판사항', '구분', '원출원번호/일자', '관련 출원번호',
       '기술이전 희망', '심사청구여부/일자', '심사청구항수', '(21) 출원번호/일자(국제)', '번역문제출일자',
       '원출원번호/일자(국제)', '제목', '출원인 ', '피인용 횟수 ', 'IPC ', '출원번호(일자) ', '최종권리자 ',
       '요약', '1', '2'],
      dtype='object')

In [435]:
all_data.iloc[461]['IPC '].replace("'",'').replace('[','').split(' ')

['G06Q',
 '30/06',
 'G06Q',
 '30/02',
 'G06Q',
 '40/02',
 'G06N',
 '20/00',
 'H04L',
 '65/40\n',
 'G06Q',
 '10/08',
 '.....]']

In [436]:
lst_IPC = []
for idx,i in enumerate(all_data['IPC ']):
    w = i.replace("'",'').replace('[','').split(' ')
    if ('G06Q' in w) & ('G06N' in w) & ('H04L' in w):
        lst_IPC.append(idx)


In [437]:
lst_IPC

[127,
 142,
 193,
 281,
 286,
 456,
 457,
 461,
 614,
 764,
 768,
 784,
 796,
 866,
 871,
 872,
 878,
 879,
 880,
 885,
 889,
 970,
 1115,
 1499,
 1867,
 1890,
 2093,
 2197,
 2234,
 2260,
 3184,
 3188,
 3208,
 3368,
 3375,
 3581,
 3747,
 4327]

In [402]:
print(all_data.iloc[8581]['제목'])
df_test.iloc[8581][0]

라인 대 라인 동기화를 달성하는 방법 및 HSYNC 지연 회로


'높은 정도의 동기화 정확도를 갖는 라인 대 라인(line-to-line) 동기화를 달성하는 방법이 제공된다. 동기화는 펠(pel) 경계들에서 PELCLK 기초 논리(PELCLK-based logic)에 스캔(sacn)들의 시작을 알리고, 펠 발생 논리와 레이저 제어 신호 사이에서 데이터를 전달할 때 서브-펠(sub-pel) 쉬프트(shift)를 제공하는 특정한 고속 논리 블록에 의해 실행된다. 동기화 에러는 단일 멀티플렉서 소자를 통한 전파 지연들만큼 적게 만들어진다. 이러한 지연들은 1/2 ns 보다 적은 값들로 제어될 수 있다. 클럭들을 중단 및 재시작할 필요가 없어진다. 복잡한 모든 펠 발생 논리는 동기화 태스크(task)와 관계없이 비교적 낮은 주파수 PELCLK으로 클럭화 된다. 동기화, 펠 경계, PELCLK, 기초 논리, 스캔, 클럭높은 정도의 동기화 정확도(a high degree of synchronization accuracy)로, 프린터에서 라인 대 라인(line-to-line) 동기화를 달성하는 방법에 있어서,펠(pel) 발생에 동기화된 펠 클럭을 제공하는 단계;복수의 지연 소자들을 포함하는 지연 체인(delay chain)을 제공하는 단계로서, 상기 복수의 지연 소자들의 각각은 데이터 출력 신호를 갖는, 상기 지연 체인을 제공하는 단계와;수평 동기화 신호(HSYNC)를 상기 지연 체인의 상기 복수의 소자들 중 첫 번째 것에 디렉팅(directing)하는 단계로서, 상기 HSYNC는 상기 펠 클럭으로부터 비동기화되는, 상기 디렉팅 단계와;상기 HSYNC가 검출된 후 상기 펠 클럭의 제 1 사이클의 발생 전에 데이터 출력 신호를 상태 변화시키는 상기 지연 체인의 상기 복수의 지연 소자들의 수를 결정하는 단계와;이전에 데이터 출력 신호를 상태 변화시키지 않았던 상기 지연 체인의 지연 소자들을 통해 비디오 데이터 입력 신호를 디렉팅하여, 상기 복수의 지연 소자들의 일부를 통해 상기 비디오 데이터 입력 신호의 위상을 쉬프트(shift)하는 단계

In [380]:
print(all_data.iloc[461]['제목'])
all_data.iloc[461]['요약']

블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템(FRANCHISE DISTRIBUTION AND MANAGEMENT SYSTEM USING BLOCKCHAIN AND MACHINE LEARNING)


'본 발명은 블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템에 관한 것으로, 이 시스템은, 상기 프랜차이즈 유통 및 관리 시스템은 프랜차이즈 협력업체 단말과 통신가능하도록 구성된 중개 관리 서버를 포함하고, 협력업체 단말은 프랜차이즈 본점측 단말, 가맹점측 단말, 물류측 단말, 및 금융사측 단말을 포함하고, 중개 관리 서버는 상품의 주문을 관리하는 주문관리를 수행하는 주문관리 모듈, 물류를 관리하는 수행하는 물류관리 모듈, 구매/생산 관리를 수행하는 구매/생산관리 모듈, 재고/손익을 관리하는 재고/손익 관리 모듈, 거래 내역을 블록체인 네트워크에 저장하는 거래내역 등재 모듈을 포함하고, 블록체인 네트워크는 거래내역이 저장되는 블록체인 주노드 네트워크와 블록체인 주노드 네트워크에 저장된 거래내역이 실시간으로 동기화되어 저장되는 블록체인 부노드 네트워크를 포함하고, 중개 관리 서버는 블록체인 주노드 네트워크에 연결되어 있고, 상기 협력업체 단말은 블록체인 부노드 네트워크에 연결되어, 중개 관리 서버는 블록체인 협력업체 사이에 거래가 발생되면 주노드 네트워크에 거래 내역을 저장하는 동시에 부노드 네트워크에 거래 내역을 복제하여 분산 저장하도록 구성된다.블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템에 있어서,프랜차이즈 협력업체 단말과 통신가능하도록 구성된 중개 관리 서버를 포함하고,상기 협력업체 단말은 프랜차이즈 본점측 단말, 가맹점측 단말,  물류측 단말, 및 금융사측 단말을 포함하고,상기 중개 관리 서버는 상품의 주문을 관리하는 주문관리를 수행하는 주문관리 모듈, 물류를 관리하는 수행하는 문류관리 모듈, 구매/생산 관리를 수행하는 구매/생산관리 모듈, 재고/손익을 관리하는 재고/손익 관리 모듈, 거래 내역을 블록체인 네트워크에 저자하는 거래내역 등재 모듈을 포함하고,상기 블록체인 네트워크는 거래내역이 저장되는 블록체인 주노드 네트워크와 블록체인 주노드 네트워크에 저장된 거래내역이 실시간으로 동기화되어 저장되는 블록체인 부노드 네트워크

In [390]:
pattern = r'\([^)]*\)'
text_2 =  re.sub(pattern=pattern, repl='', string= df_test.iloc[461].values[0]).replace('\n','')


lst_all = []
for j in [i if len(i) != 0 else  '' for i in tokenizer.tokenize(text_2)]:
    if (len(j) == 0) | ('#' in j) | ('상기' in j) | ('[' in j) | (j in stopwords[0].values) | (',' in j):
        pass
    else:
        lst_all.append(j)

text_3 = ' '.join(lst_all)

tokens=tokenizer(text_3 ,max_length=512,padding='max_length',truncation=True)

In [395]:
pd.Series(lst_all).value_counts()[pd.Series(lst_all).value_counts() > 10]

관리      28
단말      26
블록체인    24
네트워크    18
저장      14
중개      14
서버      14
포함      13
데이터     13
모듈      12
프랜      11
dtype: int64

In [388]:
pd.Series(lst_all).value_counts()

신호       20
선택       18
스캔       17
출력       16
인에이블     14
수단       12
데이터       8
입력        8
셀         7
플립플롭      7
위한        7
메모리       6
의사        6
응답        6
최종        6
테스트       5
바운더리      4
모드        4
출력단       4
시         4
폴         3
내부        3
내장        3
.         3
노말        2
구비        2
사이클       2
지연        2
디스에이블     2
통해        2
특징        2
내보내       2
본         2
클럭        2
발명        2
검출        2
블록        2
칩         2
측면        1
줄이        1
소모        1
ATP       1
높이        1
커버리지      1
연결        1
특히        1
제공        1
멀티플렉서     1
dtype: int64

In [108]:
tokens

[2, 9495, 2678, 5900, 3]